# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv = '../output_data/cities.csv'
weather_data = pd.read_csv(csv)
weather_data.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,East London,0,ZA,1592407654,59,-33.02,27.91,19.00,5.70
1,1,Albany,0,US,1592407857,42,42.60,-73.97,26.67,1.33
2,2,Lüderitz,0,NaN,1592407857,10,-26.65,15.16,31.00,3.60
3,3,La Ronge,90,CA,1592407858,87,55.10,-105.28,8.00,6.20
4,4,Naze,98,JP,1592407858,100,28.37,129.48,26.00,3.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [100]:
gmaps.configure(api_key=g_key)
weather_data


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,East London,0,ZA,1592407654,59,-33.02,27.91,19.00,5.70
1,1,Albany,0,US,1592407857,42,42.60,-73.97,26.67,1.33
3,3,La Ronge,90,CA,1592407858,87,55.10,-105.28,8.00,6.20
4,4,Naze,98,JP,1592407858,100,28.37,129.48,26.00,3.60
5,5,Funadhoo,100,MV,1592407858,80,6.15,73.27,29.15,6.42
...,...,...,...,...,...,...,...,...,...,...
535,535,Talcahuano,90,CL,1592407944,100,-36.72,-73.12,11.00,2.10
536,536,Gushikawa,1,JP,1592407944,88,26.35,127.87,29.00,6.20
537,537,Ostrovnoy,80,RU,1592407679,69,68.05,39.51,6.89,3.62
538,538,Yamada,0,JP,1592407944,83,39.47,141.95,14.86,1.53


In [116]:
# Store 'Lat' and 'Lng' into  locations 
weather_data = weather_data.dropna()
intensity = weather_data['Humidity'].max()
cent= (0, 0)
weather_data = weather_data.loc[weather_data["Max Temp"] >= 0,:]
locations = weather_data[["Lat", "Lng"]].astype(float)
heat_data = weather_data["Max Temp"].astype(float)
fig = gmaps.figure(center=cent, zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=heat_data, 
                                 dissipating=False, max_intensity=intensity, point_radius = 5)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [106]:
# Narrow down the cities to fit weather conditions
vacation_data = weather_data.loc[(weather_data["Max Temp"] >= 30) & 
                                 (weather_data["Max Temp"] < 40) &
                                 (weather_data["Humidity"] < 50) & 
                                 (weather_data["Cloudiness"] < 50) & 
                                 (weather_data["Wind Speed"] < 3)]
vacation_data = vacation_data.reset_index(drop=True)
vacation_data

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,128,Gorodishche,0,RU,1592407878,14,48.80,44.47,32.00,2.00
1,134,Humaitá,4,BR,1592407879,42,-7.51,-63.02,33.09,1.66
2,138,La Cruz,0,MX,1592407880,44,23.92,-106.90,30.56,0.89
3,154,Ojinaga,1,MX,1592407882,33,29.57,-104.42,31.00,2.60
4,229,Nocupétaro,0,MX,1592407894,43,19.05,-101.15,31.24,1.35
5,352,Barwāla,0,IN,1592407914,8,29.38,75.92,39.08,1.20
6,362,San Borja,40,BO,1592407916,46,-14.82,-66.85,30.86,2.30
7,406,Carauari,0,BR,1592407923,46,-4.88,-66.90,32.68,1.87
8,492,Kandi,27,BJ,1592407937,48,11.13,2.94,33.85,2.92
9,515,Mulegé,0,MX,1592407940,24,26.88,-111.98,32.24,1.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [107]:
hotel_df = vacation_data
hotel_df["Hotel Name"]= ""



base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 8000,
    "type": "hotel",
    "key": g_key,
}
for index, row in hotel_df.iterrows():
    latitude = str(row['Lat'])
    longitude = str(row['Lng'])
    target_coor = latitude + ", " + longitude
    params['location'] = target_coor
    #print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params)
    hotels_data = response.json()
    #print(json.dumps(hotels_data, indent=4, sort_keys=True))
    results = hotels_data['results']
    try:
        print(hotels_data["results"][0]["name"])
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Volgograd
Humaitá District
La Cruz
Ojinaga
Nocupétaro de Morelos
Barwala
Hubara
Carauari
Kandi
Mulegé


In [121]:
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,128,Gorodishche,0,RU,1592407878,14,48.80,44.47,32.00,2.00,Volgograd
1,134,Humaitá,4,BR,1592407879,42,-7.51,-63.02,33.09,1.66,Humaitá District
2,138,La Cruz,0,MX,1592407880,44,23.92,-106.90,30.56,0.89,La Cruz
3,154,Ojinaga,1,MX,1592407882,33,29.57,-104.42,31.00,2.60,Ojinaga
4,229,Nocupétaro,0,MX,1592407894,43,19.05,-101.15,31.24,1.35,Nocupétaro de Morelos
5,352,Barwāla,0,IN,1592407914,8,29.38,75.92,39.08,1.20,Barwala
6,362,San Borja,40,BO,1592407916,46,-14.82,-66.85,30.86,2.30,Hubara
7,406,Carauari,0,BR,1592407923,46,-4.88,-66.90,32.68,1.87,Carauari
8,492,Kandi,27,BJ,1592407937,48,11.13,2.94,33.85,2.92,Kandi
9,515,Mulegé,0,MX,1592407940,24,26.88,-111.98,32.24,1.55,Mulegé


In [122]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [123]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))